this is the exploratory data analysis for the BTTAI Copy.AI project
    in this notebook, i'll be looking at the EPA data through all the stuff there is to do

In [1]:
# import bs as needed
import pandas as pd
import numpy as np
import os

In [2]:
# begin by loading a csv
    # start maybe with the daily aqi by cbsa 2024, just to get this working
# df = pd.read_csv("/data/aqi/daily_aqi_by_cbsa_2023.csv", header=0)
df = pd.read_csv(os.path.join(os.getcwd(), "data", "aqi", "daily_aqi_by_cbsa_2023.csv"), header=0)
df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,"Aberdeen, SD",10100,2023-01-01,33,Good,PM2.5,46-013-0004,1
1,"Aberdeen, SD",10100,2023-01-02,56,Moderate,PM2.5,46-013-0004,1
2,"Aberdeen, SD",10100,2023-01-03,34,Good,PM2.5,46-013-0004,1
3,"Aberdeen, SD",10100,2023-01-04,10,Good,PM2.5,46-013-0004,1
4,"Aberdeen, SD",10100,2023-01-05,51,Moderate,PM2.5,46-013-0004,1


In [3]:
# yippie! it worked
# now let's find a way to trim this down to just the NYC stuff
df_nyc = df.loc[df['CBSA'] == 'New York-Newark-Jersey City, NY-NJ-PA']
df_nyc.head(10)

# now there's a file with AQI for NYC for all of 2024 so far
    # let's do the same for other types of data

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
107643,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-01,57,Moderate,PM2.5,36-059-0005,33
107644,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-02,62,Moderate,PM2.5,36-103-0009,33
107645,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-03,78,Moderate,PM2.5,36-081-0124,36
107646,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-04,70,Moderate,PM2.5,36-081-0124,33
107647,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-05,64,Moderate,PM2.5,34-039-0004,32
107648,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-06,58,Moderate,PM2.5,36-061-0115,36
107649,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-07,53,Moderate,PM2.5,36-061-0115,33
107650,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-08,54,Moderate,PM2.5,36-103-0009,31
107651,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-09,59,Moderate,PM2.5,36-061-0115,35
107652,"New York-Newark-Jersey City, NY-NJ-PA",35620,2023-01-10,60,Moderate,PM2.5,34-029-2002,32


In [4]:
# this is the wind one, can then do the same things for temp, pressure, etc
    # the files all have the headers and general format
        # most of this isn't useful for us. the main thing is the 'arithmetic mean' column and date
            # once it's been cut down to just that, it can be merged with the others into one big (well-labled!) df
    # the unit is in there somewhere, but it should be consistent
        # might be good to normalize the values at some point

# df_wind = pd.read_csv(os.path.join(os.getcwd(), "data", "meteorological", "daily_WIND_2023.csv"), header=0)
# df_wind.head()
# # get the CBSA name to narrow it down to the NYC ones
# df_wind_nyc = df_wind.loc[df_wind['CBSA Name'] == 'New York-Newark-Jersey City, NY-NJ-PA']
# df_wind_nyc.head(10)

In [5]:
# ok so that's that! now there's a general idea for what to do, which is:
# download all these files down to 2019
    # have currently skipped 2024 because it's incomplete
# trim them to just the nyc stuff
# trim everything except parameter name, date local, arithmetic mean, state name, county name, CBSA name
    # keep all the location stuff for merging
    # can safely get rid of:
        # state code, county code, site number, parameter code, POC, latitude, longitude, datum, sample duration,
        # pollutant standard, units?, event type, observation count, observation percent, 1st max value, 1st max hour,
        # AQI (usually empty anyway), method code, method name, local site name, address, date of last change
            # don't need city name, will merge on county and call it a day
            # units can just be looked up
# split into separate dfs for the separate parameters
    # some of these have various measurements in one file
# relabel 'arithmetic mean' to be more descriptive
    # should be the parameter name, once split
# merge all of them based on...
    # location?
        # maybe start with county name, get all the stuff from one city in one place
        # then maybe merge it more as needed, if the cities are sufficiently close
            # might end up discarding the non-queens stuff?
            # once there's multiple values for one date for one type of data, will have to average them in some way
                # they can't just be merged away since the CBSA/col will be the same but the data will be different
            # not sure how to go about this, but for later
     # date local?
        # this would merge various locations all into one, may or may not be what's intended
# once it's all one big file with all the possibly necessary data that the EPA could give for every year 2019-23, we're good

In [21]:
def process_file(filename):
    # access file
    df_new = pd.read_csv(filename, header=0)

    # for some reason, aqi has a different name for the column
    col_name = ''
    if 'aqi' in filename:
        col_name = 'CBSA'
    else:
        col_name = 'CBSA Name'
    
    # trim rows to just nyc
        # only rows with a "CBSA Name" of "New York-Newark-Jersey City, NY-NJ-PA"
    df_new_nyc = df_new.loc[df_new[col_name] == 'New York-Newark-Jersey City, NY-NJ-PA']
    #print(df_new_nyc.head())

    # trim columns to just useful
    # again, aqi is different
    if 'aqi' in filename:
        df_new_nyc = df_new_nyc.drop(['CBSA', 'CBSA Code', 'Defining Site', 'Number of Sites Reporting', 'Defining Parameter'], axis = 1)
    else:
        # trim everything except parameter name, date local, arithmetic mean, state name, county name, ~~city name~~, CBSA name
        df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \
                        axis = 1, inplace = True)
        df_new_nyc = df_new_nyc.rename(columns={'Date Local': 'Date'})
        df_new_nyc = df_new_nyc.rename(columns={'Arithmetic Mean': df_new_nyc.iloc[1,0]})
        df_new_nyc = df_new_nyc.drop(columns = 'Parameter Name')
    
    # return the final df
    return df_new_nyc

In [24]:
# take in the folder ("aqi") and base file name ("aqi_by_cbsa")
def process_file_set(folder, base_filename):
    whole_file_df = pd.DataFrame()
    # loop through all the years i have files for
    for i in range(2019, 2024):
        # append the year to the end of the filename, then call process_file
        filename = "daily_" + base_filename + "_" + str(i) + ".csv"

        filename = os.path.join(os.getcwd(), "data", folder, filename)

        print(filename)
        df = process_file(filename) # have this be equal to something

        # once the thing was returned, start to merge it with the a big one for this folder
        whole_file_df = pd.concat([whole_file_df, df])
        #print(whole_file_df.head())
    # return the folder one
    return whole_file_df

In [25]:
# start making a function to do all of the stuff for each file
df_full = pd.DataFrame()

# do all of these then merge into the big big one
aqi = process_file_set('aqi', 'aqi_by_cbsa')
carbon = process_file_set('criteria_gases', '42101')
carbon2 = process_file_set('criteria_gases', '42401')

new_big_file_df = pd.merge(aqi, carbon, how='outer', on='Date')
new_big_file_df = pd.merge(new_big_file_df, carbon2, how='outer', on=['Date', 'County Name'])
new_big_file_df = new_big_file_df.drop_duplicates(subset=['Date', 'County Name'])
new_big_file_df.to_csv('please_work_fr.csv', sep=',', index=False, encoding='utf-8')

/Users/soarderius/epa/data/aqi/daily_aqi_by_cbsa_2019.csv
/Users/soarderius/epa/data/aqi/daily_aqi_by_cbsa_2020.csv
/Users/soarderius/epa/data/aqi/daily_aqi_by_cbsa_2021.csv
/Users/soarderius/epa/data/aqi/daily_aqi_by_cbsa_2022.csv
/Users/soarderius/epa/data/aqi/daily_aqi_by_cbsa_2023.csv
/Users/soarderius/epa/data/criteria_gases/daily_42101_2019.csv


/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(filename, header=0)
/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \


/Users/soarderius/epa/data/criteria_gases/daily_42101_2020.csv


/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(filename, header=0)
/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \


/Users/soarderius/epa/data/criteria_gases/daily_42101_2021.csv


...

/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(filename, header=0)
/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \


/Users/soarderius/epa/data/criteria_gases/daily_42401_2022.csv


/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(filename, header=0)
/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \


/Users/soarderius/epa/data/criteria_gases/daily_42401_2023.csv


/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(filename, header=0)
/var/folders/v4/zyvy1vmn39ddql58q2r98k8h0000gn/T/ipykernel_89514/272343036.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_nyc.drop(df_new_nyc.columns.difference(['Parameter Name','Date Local', 'Arithmetic Mean', 'County Name']), \
